<a href="https://colab.research.google.com/github/Techneider/RedNeuronal_PrediccionCancerDeMama/blob/main/RedNeuronal_PrediccionCancerDeMama_Techneider_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importación de TensorFlow (recordar que pip es una herramienta y no un comando, por lo que debe ir pecedido de "!")

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.0


Se ha cerrado la conexión y reestablecido a travé de GPU, para importar Tensorflow para GPU.

In [ ]:
!pip install tensorflow-gpu


In [ ]:
import tensorflow as tf
print(tf.__version__)


2.4.0


**Para este estudio con red neuronal, se utilizarán los datos publicados por el Doctor William H. Wolberg del University of Wisconsin Hospitals. **
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

La base de datos origen dispone de más de 600 registros. Cada registro es el analisis de una muestra de celulas, y continene 30 valores de mediciones de ciertas caracteristicas analizadas, más un valor que indica resultado de si formaron un cancer benigno o maligno.

La base de datos origen se ha tratado previamente de la siguiente forma:
1. Los registros se han distribuido a su vez en 2 bases de datos.csv 
- La primera con más de 400 registros para el training de la red neuronal 
- La segunda com más de 150 registros para el test del entrenamiento.
2. Cada una de las bases anteriores, training y test, se ha separado en 2 partes:
- Parte X, contiene las mediciones = valores X = entrada de la red neuronal.
- Parte Y, contiene el valor resultado = valor Y = salida o resultado (0 = benigno, 1= maligno).


Finalement, como resultado de este tratamiento, se obtienen 4 archivos .csv (training X, training Y, test X, test Y).


Se cargan los datos .csv en pandas, que es una herramienta pandas para manejar **Dataframe**, que es como le llama al conjunto de row+columns (tabla de datos).


In [ ]:
import pandas as pd

from google.colab import files
file = files.upload()
X_train = pd.read_csv("xtrain.csv", header=None)
Y_train = pd.read_csv("ytrain.csv", header=None)
X_test = pd.read_csv("xtest.csv", header=None)
Y_test = pd.read_csv("ytest.csv", header=None)

Saving xtest.csv to xtest.csv
Saving xtrain.csv to xtrain.csv
Saving ytest.csv to ytest.csv
Saving ytrain.csv to ytrain.csv


Se utiliza Keras en la API **Sequential** para crear la Red Neuronal que va a ser entrenada con estos datos. 
Esta Red Neuronal será empleada para predecir futuros valores que alimenten la presencia de cancer (0 = benigno, 1 = maligno).

Formaremos una Red neuronal compuesta de 4 capas. Las 3 primeras capas con **funcion de transformación lineal rectificada (relu)** que evitarán la saturación de la funcion **sigmoidea** activada en la última capa. 
Esta función sigmoidea será la que clasifique todos los datos de entrada en 0 ó 1.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential() #Inicializando la Red Neuronal (ANN)

classifier.add(Dense(units = 16, activation ='relu', input_dim = 30)) # Capa de 16 nodos, activación relu y entrada de dimension 30 = nº de mediciones
classifier.add(Dense(units = 8, activation = 'relu')) # Capa de 8 nodos de activación relu
classifier.add(Dense(units = 6, activation = 'relu')) # Capa de 6 nodos de activación relu
classifier.add(Dense(units = 1, activation = 'sigmoid')) # Capa final de 1 nodo activada por una función sigmoidea


Establecida la Red Neuronal, se especifica el **Optimizador** y la **Función de Perdida**.

La función de perdida medirá en cada iteración que tan bien fue el entreamiento, y obtendrá un resultado como error de aproximación, para luego aplicar el optimizador e intentar reducir el error. 

In [ ]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

Se entrena la Red Neuronal usando la función **classifier.predict**, y se le pasan los valores de las X y de las Y, así como el numero muestras que componen el lote (**batch_size**) en el que se va a iterar (**epochs**) el optimizador para aproximar sus resultados, con el fin de establecer una relación que permita constuir la Red Neuronal que replique esos datos.

In [ ]:
classifier.fit(X_train, Y_train, batch_size = 1, epochs = 100)

Epoch 1/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0018
Epoch 2/100
455/455 [==============================] - 1s 2ms/step - loss: 7.7483e-04
Epoch 3/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0021
Epoch 4/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 5/100
455/455 [==============================] - 1s 2ms/step - loss: 3.9922e-04
Epoch 6/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0100
Epoch 7/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0039
Epoch 8/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0020
Epoch 9/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0073
Epoch 10/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0015
Epoch 11/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0015
Epoch 12/100
455/455 [==============================] - 1s 2ms/step - loss: 0.0016
Epoch

Despues de esto, la Red Neuronal habrá mejorado su conjetura.
Es hora de predecir nuevos valores con esta Red, usando **classifier.predict** y pasandole los valores X que aislamos para realizar el test, y que la red desconoce. 

El resultado será una predicción en forma de probabilidad, por lo que deberemos limpiarla diciendo que si la probabilidad es mayor de 0.5 establezca 1 y si es menor establezca 0.

In [ ]:
Y_pred = classifier.predict(X_test)
Y_pred = [1 if y>=0.5 else 0 for y in Y_pred ]

Finalmente comprobamos que tal ha sido esta predicción, creando un breve codigo que se encargará de contrastar los valores Y_pred obtenidos en la predicción, contra los valores reales que habiamos aislado en la tabla Y_test.

Cada fallo de coincidencia entre el valor de **Y_pred** e **Y_test**, incrementará el valor de la variable **fallos**, indicandonos así cuantos errores de predicción ha cometido nuestra Red-Neuronal.

In [ ]:
total = 0
right = 0
wrong = 0
for i in Y_pred:
  total = total +1
  if(Y_test.at[i,0] == Y_pred[i]): # Comparación de la tabla Y_Test con el valor en Y_pred
    right = right+1
  else:
    wrong=wrong+1
print("Total = " + str(total))
print("Right = " + str(right))
print("Wrong = " + str(wrong))

Total = 114
Right = 114
Wrong = 0
